# Ch5: Introduction To Modern Symmetric Key Ciphers
## 5.1 Permutation Box 
A Permutation Box is a transposition Cipher that takes in a permutation and reorders the bits or a block of data in as per the permutation. There can be 3 types of PBox's:

1. Straigt P-Box (is invertible)
2. Compression P-Box (is non-invertible)
3. Expansion P-Box (is non-invertible)

We define a class `PBox` and define the permutation mapping for all 3 types of P-Boxes using a python dictionary.

In [5]:
class PBox:
    def __init__(self, key: dict):
        self.key = key
        self.in_degree = len(key)
        self.out_degree = sum(len(value) if isinstance(value, list) else 1 for value in key.values())

    def __repr__(self) -> str:
        return 'PBox' + str(self.key)

    def permutate(self, sequence: list) -> list:
        result = [0] * self.out_degree
        for index, value in enumerate(sequence):
            if (index + 1) in self.key:
                indices = self.key.get(index + 1, [])
                indices = indices if isinstance(indices, list) else [indices]
                for i in indices:
                    result[i - 1] = value
        return result

    def is_invertible(self) -> bool:
        return self.in_degree == self.out_degree

    def invert(self):
        if self.is_invertible():
            result = {}
            for index, mapping in self.key.items():
                result[mapping] = index
            return PBox(result)

In [6]:
# Compression P-Box
compression_p_box = PBox({1: 1, 2: [], 3: 2})
print('In Degree:', compression_p_box.in_degree)
print('Out Degree:', compression_p_box.out_degree)
print(compression_p_box.permutate([10, 20, 30]))

In Degree: 3
Out Degree: 2
[10, 30]


In [7]:
# Compression boxes are non-invertible
print(compression_p_box.is_invertible())

False


In [8]:
# Expansion P Box
expansion_p_box = PBox({1: 1, 2: 2, 3: [3, 4]})
print(expansion_p_box)
print('In Degree:', expansion_p_box.in_degree)
print('Out Degree:', expansion_p_box.out_degree)
print(expansion_p_box.permutate([10, 20, 30]))

PBox{1: 1, 2: 2, 3: [3, 4]}
In Degree: 3
Out Degree: 4
[10, 20, 30, 30]


In [9]:
# Expansion P Boxes are non invertible
print(expansion_p_box.is_invertible())

False


In [11]:
# Straight P Boxes
p_box = PBox({1: 3, 2: 1, 3: 2})
print('In Degree:', p_box.in_degree)
print('Out Degree:', p_box.out_degree)
print(p_box.permutate([10, 20, 30]))

In Degree: 3
Out Degree: 3
[20, 30, 10]


In [13]:
# straight P boxes are invertible
print(p_box.is_invertible())
print('Inverse:', p_box.invert())

True
Inverse: PBox{3: 1, 1: 2, 2: 3}


In [12]:
# composition of pemutation and inverse leads in the identity permutations
print(p_box.invert().permutate(p_box.permutate([10, 20, 30])))

[10, 20, 30]


## 5.2 Feistel Cipher
The Feistel Cipher is a product cipher and uses many different coomponents, even non-invertible components in the cipher. The non-invertible part is called the __mixer__ and uses the _XOR_ function to encrypt and decrypt a number using a secret key (also a number).

In [14]:
class FiestelMixerCipher:
    def __init__(self, key: int):
        self.key = key

    def encrypt(self, number: int) -> int:
        return number ^ self.key

    def decrypt(self, number: int) -> int:
        return self.encrypt(number)

In [ ]:
fiestel_mixer = FiestelMixerCipher(9)
ciphertext = fiestel_mixer.encrypt(10)
print(ciphertext)